# SARIMAX GDP Forecasting System

## Overview

This notebook demonstrates a comprehensive SARIMAX (Seasonal AutoRegressive Integrated Moving Average with eXogenous variables) modeling framework for GDP forecasting. The system has been refactored into modular components for better maintainability and extensibility.

### Key Features:
- **Multi-country GDP forecasting** (US, EU, China)
- **Exogenous variable integration** (PMI, ESI, other economic indicators)
- **Automated hyperparameter optimization** via grid search
- **Comprehensive evaluation metrics** and statistical tests
- **Rolling-origin validation** for realistic forecast assessment
- **Professional visualization** and diagnostic tools

## Mathematical Foundation

### SARIMAX Model Specification

The SARIMAX model combines seasonal and non-seasonal components with exogenous variables:

$$\phi(B)\Phi(B^s)(1-B)^d(1-B^s)^D y_t = \theta(B)\Theta(B^s)\varepsilon_t + \beta^T X_t$$

Where:
- $\phi(B) = 1 - \phi_1 B - \phi_2 B^2 - \ldots - \phi_p B^p$ (non-seasonal AR polynomial)
- $\Phi(B^s) = 1 - \Phi_1 B^s - \Phi_2 B^{2s} - \ldots - \Phi_P B^{Ps}$ (seasonal AR polynomial)
- $\theta(B) = 1 + \theta_1 B + \theta_2 B^2 + \ldots + \theta_q B^q$ (non-seasonal MA polynomial)
- $\Theta(B^s) = 1 + \Theta_1 B^s + \Theta_2 B^{2s} + \ldots + \Theta_Q B^{Qs}$ (seasonal MA polynomial)
- $B$ is the backshift operator: $B y_t = y_{t-1}$
- $d, D$ are differencing orders (non-seasonal, seasonal)
- $s$ is the seasonal period (4 for quarterly data)
- $X_t$ is the vector of exogenous variables
- $\varepsilon_t \sim N(0, \sigma^2)$ is white noise

### Model Order Notation

The model is denoted as **SARIMAX(p,d,q)(P,D,Q,s)** where:
- $(p,d,q)$: non-seasonal components (AR order, differencing, MA order)
- $(P,D,Q,s)$: seasonal components (seasonal AR, seasonal differencing, seasonal MA, period)

### Time Series Transformations

#### 1. Level (Identity)
$$y_t^{(level)} = y_t$$

#### 2. Quarter-over-Quarter (QoQ)
$$y_t^{(qoq)} = 100 \times \frac{y_t - y_{t-1}}{y_{t-1}} = 100 \times \left(\frac{y_t}{y_{t-1}} - 1\right)$$

#### 3. Year-over-Year (YoY)
$$y_t^{(yoy)} = 100 \times \frac{y_t - y_{t-4}}{y_{t-4}} = 100 \times \left(\frac{y_t}{y_{t-4}} - 1\right)$$

#### 4. Log Difference
$$y_t^{(log\_diff)} = 100 \times (\log y_t - \log y_{t-1}) = 100 \times \log\left(\frac{y_t}{y_{t-1}}\right)$$

The log difference approximates percentage change for small changes: $\log(1+x) \approx x$ when $|x| << 1$.

### Stationarity Testing

#### Augmented Dickey-Fuller (ADF) Test

The ADF test examines the null hypothesis of a unit root:

$$\Delta y_t = \alpha + \beta t + \gamma y_{t-1} + \sum_{i=1}^{p} \delta_i \Delta y_{t-i} + \varepsilon_t$$

Where:
- $H_0: \gamma = 0$ (unit root exists, series is non-stationary)
- $H_1: \gamma < 0$ (no unit root, series is stationary)

**Test Statistic:**
$$ADF = \frac{\hat{\gamma}}{SE(\hat{\gamma})}$$

The test statistic follows a non-standard distribution. Critical values depend on sample size and model specification.

### Model Selection Criteria

#### Akaike Information Criterion (AIC)
$$AIC = -2\log(L) + 2k$$

#### Bayesian Information Criterion (BIC)
$$BIC = -2\log(L) + k\log(n)$$

#### Hannan-Quinn Information Criterion (HQIC)
$$HQIC = -2\log(L) + 2k\log(\log(n))$$

Where:
- $L$ is the maximized likelihood
- $k$ is the number of parameters
- $n$ is the sample size

**Model Selection Rule:** Choose model with minimum IC value.

## Setup and Imports

Let's start by importing the necessary modules and setting up our environment:

In [ ]:
# Core imports
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import product
import seaborn as sns

# Import our refactored modules
from gdp_forecaster_src.config_utils import initialize_config
from gdp_forecaster_src.data_utils import load_macro_data, load_gdp_series_csv
from gdp_forecaster_src.parsing_utils import parse_range_arg, parse_intervals_arg
from gdp_forecaster_src.transform_utils import apply_target_transform
from gdp_forecaster_src.metrics_utils import compute_metrics
from gdp_forecaster_src.plotting_utils import plot_realgdp, plot_forecast_comparison
from gdp_forecaster_src.forecasting_utils import (
    optimize_sarimax, rolling_one_step_predictions, adf_test, 
    fit_best_sarimax_model, validate_sarimax_inputs
)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure matplotlib and seaborn
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 12

# Initialize configuration
initialize_config()

print("✅ All modules imported successfully")
print(f"✅ Working directory: {Path.cwd()}")

## Data Loading and Exploration

### Loading US Macroeconomic Data

We'll start with the classic US macroeconomic quarterly dataset from statsmodels, covering 1959-2009:

In [ ]:
# Load the macro dataset
data_path = Path("data/us_macro_quarterly.csv")
df_macro = load_macro_data(data_path)

print(f"Dataset shape: {df_macro.shape}")
print(f"Time period: {df_macro['year'].min()}-{df_macro['year'].max()}")
print(f"\nColumns: {list(df_macro.columns)}")

# Display first few rows
df_macro.head()

In [ ]:
# Visualize the Real GDP series
plot_realgdp(df_macro, Path('figures/notebook_realgdp.png'))
from IPython.display import Image
Image('figures/notebook_realgdp.png')

### Stationarity Analysis

Before modeling, we need to assess stationarity using the Augmented Dickey-Fuller test:

**Null Hypothesis:** $H_0: \gamma = 0$ (series has unit root, non-stationary)

**Alternative Hypothesis:** $H_1: \gamma < 0$ (series is stationary)

In [ ]:
# Extract GDP series and test
gdp_series = df_macro['realgdp']
adf_stat_level, adf_pval_level = adf_test(gdp_series)
gdp_diff = gdp_series.diff().dropna()
adf_stat_diff, adf_pval_diff = adf_test(gdp_diff)

print(f"ADF p-value (level): {adf_pval_level:.4f} -> {'Stationary' if adf_pval_level < 0.05 else 'Non-stationary'}")
print(f"ADF p-value (diff): {adf_pval_diff:.4f} -> {'Stationary' if adf_pval_diff < 0.05 else 'Non-stationary'}")

## SARIMAX Model Setup and Grid Search

### Hyperparameter Optimization

We perform grid search over the hyperparameter space $(p,q,P,Q)$ to minimize AIC.

In [ ]:
target_transform = 'level'
endog, d_suggested = apply_target_transform(gdp_series, target_transform)
endog_clean = endog.dropna()
validate_sarimax_inputs(endog_clean)

p_range = parse_range_arg("0-2")
q_range = parse_range_arg("0-2")
P_range = parse_range_arg("0-1")
Q_range = parse_range_arg("0-1")
param_combinations = list(product(p_range, q_range, P_range, Q_range))

result_df = optimize_sarimax(endog=endog_clean, exog=None, order_list=param_combinations, d=d_suggested, D=0, s=4)

best_order = result_df.iloc[0]['(p,q,P,Q)']
print(f"Best model order: {best_order}")
result_df.head()

### Model Fitting and Diagnostics


In [ ]:
best_fitted = fit_best_sarimax_model(endog=endog_clean, exog=None, best_order=best_order, d=d_suggested, D=0, s=4)
print(best_fitted.summary())

In [ ]:
diag_fig = best_fitted.plot_diagnostics(figsize=(16, 10))
plt.tight_layout()
plt.show()

## Forecast Evaluation

### Rolling-Origin Validation

We implement **rolling-origin validation** to simulate real-world forecasting.

In [ ]:
TEST_HORIZON = 16
train_size = len(endog_clean) - TEST_HORIZON
coverage_levels = parse_intervals_arg("80,95")

rolling_preds, conf_bounds = rolling_one_step_predictions(
    endog=endog_clean, exog_model=None, best_order=best_order,
    d=d_suggested, D=0, s=4, coverage_levels=coverage_levels,
    start_index=train_size
)

y_test = endog_clean.iloc[train_size:]
y_train = endog_clean.iloc[:train_size]
naive_preds = [endog_clean.iloc[i-1] for i in range(train_size, len(endog_clean))]

metrics_sarimax = compute_metrics(y_test, rolling_preds, naive_preds, y_train)
metrics_naive = compute_metrics(y_test, naive_preds, naive_preds, y_train)

metrics_df = pd.DataFrame([metrics_sarimax, metrics_naive], index=['SARIMAX', 'Naive'])
metrics_df.round(4)

In [ ]:
forecasts = {"SARIMAX": rolling_preds, "Naive": naive_preds}
plot_forecast_comparison(y_test, forecasts, Path('figures/notebook_forecast_comparison.png'))
Image('figures/notebook_forecast_comparison.png')